In [102]:
import pandas as pd
import numpy as np
import econtools.metrics as mt
from itertools import *

In [198]:
def treat_dummy(df):
    
    df = pd.get_dummies(df,columns=["treat"])
    df["treat"] = df["treat_Treatment"]
    df = df.drop(columns=["treat_Control"])
    df = df.drop(columns=["treat_Treatment"])
    
    return df

def col_names(df, name):

    df["ds_"] = df["ds_" + name]
    df["ds_educ"] = df["ds_" + name]*df["educ_ave"]
    df["treat_ds_"] = df["treat_ds_"+name]
    df["treat_ds_educ"] = df["treat_ds_"+name]*df["educ_ave"]
    df["treat_educ_used"] = df["treat"]*df["educ_ave"]
    df["perf_"] = df[name]
    df["perf_educ"] = df[name]*df["educ_ave"]
    df["treat_perf"] = df["treat_"+name]
    df["treat_perf_educ"] = df["treat_"+name]*df["educ_ave"]
    
    if name != "ave" and name != "math_engl":
        df["wb_"+name] = df["wb_"+name]*100
    
    return df

#reg variables names
def reg_var(df):
    def control(ctrl):
        dummies = []
        nonlocal df
        ctrl_val  = df[ctrl].value_counts().keys().sort_values()
        for i in ctrl_val:
            dummies.append(ctrl+"_"+str(i))
        return dummies
    
    ctrl_col = ["std","perf_gap_used","C1_strat"]
    result = list(map(control,ctrl_col))
    result = list(chain.from_iterable(result))
    
    X = ["treat_perf", "treat", "perf_", "educ_ave_reg", "educ_ave_miss", "female_reg", "female_miss", "primary_resp_fem"] + result
    
    df = pd.get_dummies(df,columns = ctrl_col)
    
    return df, X

def reg_var5(df):
    def control(ctrl):
        dummies = []
        nonlocal df
        ctrl_val  = df[ctrl].value_counts().keys().sort_values()
        for i in ctrl_val:
            dummies.append(ctrl+"_"+str(i))
        return dummies
    
    ctrl_col = ["std","perf_gap_used","C1_strat"]
    result = list(map(control,ctrl_col))
    result = list(chain.from_iterable(result))
    
    X = ["educ_ave_reg", "educ_ave_miss", "female_reg", "female_miss", "primary_resp_fem"]+result
    
    df = pd.get_dummies(df,columns = ctrl_col)
    
    return df, X

def neg(df, col, sub):
    for i in col:
        if i == "":
            df[sub] = -df[sub]
        df[i+sub] = -df[i+sub]
    return df

In [207]:
#col(1)
df = pd.read_stata("data/output_data/attendance_working.dta")

df = treat_dummy(df)
df = col_names(df, "ave")
df, X = reg_var(df)

result1 = mt.reg(df, "u_ave", X , fe_name = "school_code", cluster="hhid").summary.drop(index=controls)
result1

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,coeff,se,t,p>t,CI_low,CI_high
treat_perf,0.420033,0.026267,15.990731,5.393794e-55,0.368527,0.471540
treat,-26.933979,1.378693,-19.535884,1.831089e-79,-29.637413,-24.230545
perf_,0.305571,0.018577,16.449262,6.096796e-58,0.269145,0.341997


In [159]:
#col2
df = pd.read_stata("data/output_data/wb_math_working.dta")

df = treat_dummy(df)
df = col_names(df, "math")
df, X = reg_var(df)

result2 = mt.reg(df, "wb_math", X , fe_name = "school_code", cluster="hhid").summary.drop(index=controls)
result2

,coeff,se,t,p>t,CI_low,CI_high
treat_perf,1.332674,0.093037,14.324056,7.251447e-45,1.150240,1.515108
treat,-90.841934,4.906566,-18.514359,4.976733e-72,-100.463060,-81.220808
perf_,0.656697,0.065690,9.996863,4.049521e-23,0.527887,0.785507


In [161]:
#col3 English
df = pd.read_stata("data/output_data/wb_engl_working.dta")

df = treat_dummy(df)
df = col_names(df, "engl")
df, X = reg_var(df)

result3 = mt.reg(df, "wb_engl", X , fe_name = "school_code", cluster="hhid").summary.drop(index=controls)
result3

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,coeff,se,t,p>t,CI_low,CI_high
treat_perf,1.228290,0.095764,12.826247,1.429826e-36,1.040510,1.416070
treat,-67.259738,4.841771,-13.891557,2.169933e-42,-76.753810,-57.765665
perf_,0.773028,0.073680,10.491721,2.956435e-25,0.628551,0.917504


In [188]:
#col4
df = pd.read_stata("data/output_data/textbook_working.dta")

df = treat_dummy(df)
#df["ln10p_wtp_engl_math"] = df["ln10p_wtp_engl"] - df["ln10p_wtp_math"] #do not use it..why is it here?
df = neg(df, ["ds_","treat_","treat_ds_",""],"math_engl")
df = col_names(df, "math_engl")
df, X = reg_var(df)

result4 = mt.reg(df, "ln10p_wtp_math_engl", X , fe_name = "school_code", cluster="hhid").summary.drop(index=controls)
result4

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,coeff,se,t,p>t,CI_low,CI_high
treat_perf,0.014396,0.002204,6.531872,7.772790e-11,0.010074,0.018718
treat,0.143667,0.040852,3.516755,4.442669e-04,0.063561,0.223773
perf_,-0.001487,0.001667,-0.892030,3.724588e-01,-0.004757,0.001782


In [206]:
#col5
df = pd.read_stata("data/output_data/Ability_Clean_2.dta")
df = df.rename(columns={'treat_ave':'treat_perf', 'ave':'perf_'})
df, ctrl = reg_var5(df)

X = ["treat_perf", "perf_", "high_perf_sib"] + ctrl

result5 = mt.reg(df, "tix", X , cluster="hhid").summary.drop(index=controls)
result5

C:\Users\thxkn\anaconda3\lib\site-packages\econtools\metrics\core.py:203: RuntimeWarning: invalid value encountered in sqrt
  se = pd.Series(np.sqrt(np.diagonal(vce)), index=vce.columns)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
C:\Users\thxkn\anaconda3\lib\site-packages\scipy\stats\_distn_infrastructure.py:1827: RuntimeWarning: invalid value encountered in greater_equal
  cond2 = (x >= np.asarray(_b)) & cond0


,coeff,se,t,p>t,CI_low,CI_high
treat_perf,0.042129,0.004326,9.739162,4.818448e-22,0.033647,0.050611
perf_,-0.141857,0.007593,-18.682789,3.088212e-73,-0.156745,-0.126968
high_perf_sib,0.117629,0.192268,0.611796,5.407256e-01,-0.259384,0.494642
